### Desafío Descarga de Datos desde APIs Públicas

Para este trabajo se usará la API BALLDONTLIE que contiene datos sobre los jugadores de NBA.

In [20]:
import requests
import pandas as pd
import time

# Función mejorada para realizar una solicitud GET a la API de balldontlie
def get_balldontlie(endpoint, params=None):
    base_url = "https://www.balldontlie.io/api/v1/"
    url = f"{base_url}{endpoint}"
    
    try:
        # Realizar la solicitud GET
        response = requests.get(url, params=params)
        
        # Verificar el estado de la respuesta
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error {response.status_code}: {response.text}")
    except Exception as e:
        print(f"Error en la solicitud: {str(e)}")
    
    # Devuelve un diccionario vacío en caso de error
    return {}

# Función para obtener todos los jugadores
def get_all_players():
    all_players = []
    page = 1
    
    while True:
        params = {"page": page}
        players_data = get_balldontlie("players", params)
        
        # Verificar si hay más páginas
        if not players_data or not players_data.get('meta', {}).get('next_page'):
            break
        
        # Agregar jugadores a la lista
        all_players.extend(players_data['data'])
        
        # Moverse a la siguiente página
        page += 1
        
        # Esperar un momento antes de la siguiente solicitud para evitar problemas con la API
        time.sleep(2)
    
    return all_players

# Obtener todos los jugadores
all_players_data = get_all_players()

# Crear un DataFrame con los datos de los jugadores
df_players = pd.DataFrame(all_players_data)

# Imprimir las primeras filas del DataFrame
print(df_players.head())


   id first_name  height_feet  height_inches last_name position  \
0  14        Ike          NaN            NaN  Anigbogu        C   
1  25        Ron          NaN            NaN     Baker        G   
2  47     Jabari          NaN            NaN      Bird        G   
3  67    MarShon          NaN            NaN    Brooks        G   
4  71    Lorenzo          NaN            NaN     Brown        G   

                                                team  weight_pounds  
0  {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...            NaN  
1  {'id': 20, 'abbreviation': 'NYK', 'city': 'New...            NaN  
2  {'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...            NaN  
3  {'id': 15, 'abbreviation': 'MEM', 'city': 'Mem...            NaN  
4  {'id': 28, 'abbreviation': 'TOR', 'city': 'Tor...            NaN  


In [23]:
# Obtener el número total de filas en el DataFrame
num_filas = df_players.shape[0]
num_columnas = df_players.shape[1]

# Imprimir el número total de filas
print(f"El DataFrame tiene {num_filas} filas.")
print(f"El DataFrame tiene {num_columnas} columnas.")


El DataFrame tiene 5200 filas.
El DataFrame tiene 8 columnas.


LIMPIEZA DE DATOS

In [25]:
# Contar los NaN en la columna "height_feet"
nan_counthf = df_players['height_feet'].isna().sum()
nan_counthi = df_players['height_inches'].isna().sum()
nan_countw = df_players['weight_pounds'].isna().sum()

# Imprimir el número de NaN en la columna "height_feet"
print(f"El número de NaN en la columna 'height_feet' es: {nan_counthf}")
print(f"El número de NaN en la columna 'height_inches' es: {nan_counthi}")
print(f"El número de NaN en la columna 'weight_pounds' es: {nan_countw}")


El número de NaN en la columna 'height_feet' es: 4721
El número de NaN en la columna 'height_inches' es: 4721
El número de NaN en la columna 'weight_pounds' es: 4721


Estas 3 características no me aportan nada, así que las borraré del df

In [26]:
# Eliminar las columnas 'height_feet', 'height_inches' y 'weight_pounds'
df_players = df_players.drop(['height_feet', 'height_inches', 'weight_pounds'], axis=1)

# Imprimir el DataFrame actualizado
print(df_players.head())


   id first_name last_name position  \
0  14        Ike  Anigbogu        C   
1  25        Ron     Baker        G   
2  47     Jabari      Bird        G   
3  67    MarShon    Brooks        G   
4  71    Lorenzo     Brown        G   

                                                team  
0  {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...  
1  {'id': 20, 'abbreviation': 'NYK', 'city': 'New...  
2  {'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...  
3  {'id': 15, 'abbreviation': 'MEM', 'city': 'Mem...  
4  {'id': 28, 'abbreviation': 'TOR', 'city': 'Tor...  


En mi notebook del proyecto de NBA tengo una característica llamada Player, no está dividido en nombre y apellido, así que tienen que concatenarse en una sola columna. 

In [30]:
# Concatenar 'first_name' y 'last_name' usando el operador +
df_players['Player'] = df_players['first_name'] + ' ' + df_players['last_name']

# Imprimir el DataFrame con la nueva columna 'Player'
print(df_players[['first_name', 'last_name', 'Player']])


     first_name  last_name            Player
0           Ike   Anigbogu      Ike Anigbogu
1           Ron      Baker         Ron Baker
2        Jabari       Bird       Jabari Bird
3       MarShon     Brooks    MarShon Brooks
4       Lorenzo      Brown     Lorenzo Brown
...         ...        ...               ...
5195      Micah     Potter      Micah Potter
5196        Dru      Smith         Dru Smith
5197     MarJon  Beauchamp  MarJon Beauchamp
5198      Dyson    Daniels     Dyson Daniels
5199      Jaden      Hardy       Jaden Hardy

[5200 rows x 3 columns]


El siguiente paso es unir esta tabla con la tabla del df usado en el notebook del proyecto final para ver si es que coinciden las posiciones de los jugadores y los equipos donde juegan. 